Script para descargar los avatares (fotos de perfil) de los seguidores de Twitter usando la API.

Guardar en el .env el ID del usuario del cual se quieren obtener los seguidores.

Nota> la API tiene un limite de llamadas, creo que 15 cada 15 minutos.

In [9]:
# Importa librerias
import os
import requests
import json
import time
from dotenv import load_dotenv

# Carga las variables de entorno desde el archivo .env
load_dotenv()

# Accede a los valores de las variables de entorno
user = os.getenv('USER')
token_bearer = os.getenv('TOKEN_BEARER')

In [10]:
# Función para obtener hasta 1000 seguidores, de la API
def more_data(user, next_token):
    url = 'https://api.twitter.com/2/users/' + user + '/followers?user.fields=profile_image_url&max_results=1000'
    
    # Si hay un next_token (pagina siguiente) lo agrega a la URL
    if next_token != 0:
        url += '&pagination_token=' + next_token
    
    # Define los encabezados de la solicitud con el token de autenticación
    headers = {
        'Authorization': f'Bearer {token_bearer}'
    }
    
    # Realiza la solicitud HTTP con los encabezados
    response = requests.get(url, headers = headers)
    time.sleep(3)
    
    # Verifica el código de estado de la respuesta
    if response.status_code == 200:
        
        # Accede a los datos en formato JSON
        data = response.json()             
        print(f'Datos obtenidos. Código de estado: {response.status_code}')
        return data
   
    else:
        
        print(f'Error en la solicitud. Código de estado: {response.status_code}')
        return

In [11]:
# Realiza la primera conexión, y las siguiente mientras haya next_token
print('Ejecutando primera conexión')
data = more_data(user, 0)
seguidores = []

while True:
    if 'next_token' in data['meta']:
        # respuesta = input('Hay más datos disponibles, desea descargarlos? (S/N): ')
        # if respuesta.lower() == 's':
        seguidores.extend(data['data'])
        data = more_data(user, data['meta']['next_token'])
        # else:
            # break
    else:
        seguidores.extend(data['data'])
        print('Se descargaron todos los datos disponibles.')
        break

# while 'next_token' in data['meta']:
#   seguidores.extend(data['data'])
#   data = more_data(user, data['meta']['next_token'])
#   print('Se descargaron todos los datos disponibles.')

print('Total de seguidores obtenidos:', len(seguidores))

Ejecutando primera conexión
Datos obtenidos. Código de estado: 200
Datos obtenidos. Código de estado: 200
Datos obtenidos. Código de estado: 200
Se descargaron todos los datos disponibles.
Total de seguidores obtenidos: 3000


In [13]:
# Esta pastilla no es necesaria, pero sirve para guardar el JSON

# Define el nombre de archivo para guardar los datos JSON
nombre_archivo = 'datos.json'

# Guarda los datos en el archivo JSON
with open(nombre_archivo, 'w') as archivo:
    json.dump(seguidores, archivo)

print(f'Los datos se han guardado en el archivo "{nombre_archivo}"')

Los datos se han guardado en el archivo "datos.json"


In [15]:
from os import path

# Crea la carpeta donde se van a descargar las imagenes
carpeta_salida = 'fotos/'

if path.exists(carpeta_salida) == False:
    os.mkdir(carpeta_salida)

In [29]:
# Crea el dataframe con username, link de imagen, id y nombre de cada seguidor

import pandas as pd
import numpy as np
import urllib.request, urllib.error
from IPython.display import clear_output
import shutil

from tqdm import tqdm

# # Abre el archivo JSON creado en la celda anterior
# with open('datos.json', encoding = 'utf-8') as f:
#   data = json.load(f)

# Crea el dataframe y le agrega una columna para marcar errores
df = pd.DataFrame(seguidores)
df['error'] = 0

In [34]:
# Script para mostrar el número de imagen de un usuario
df[df['username'] == 'MiltonFriedom5'].index[0]

1487

In [33]:
# Funcion que descarga cada imagen de Twitter
# i: numero de imagen / url: direccion web / ruta: carpeta donde se guardan las imagenes
def url_to_imagen(i, url, ruta):
    # guarda las extensiones jpg o jpeg como jpg
    if url[-3:] == 'jpg' or url[-4:] == 'jpeg' or url[-3:] == 'JPG' or url[-4:] == 'JPEG':
        archivo = '{}.jpg'.format(str(i).zfill(6))
        guardado = '{}{}'.format(ruta, archivo)
    # si es png lo guarda como png
    elif url[-3:] == 'png' or url[-3:] == 'PNG':
        archivo = '{}.png'.format(str(i).zfill(6))
        guardado = '{}{}'.format(ruta, archivo)
    # si es cualquier otra extension (poco probable) o no tiene link, usa el avatar por defecto
    else:
        archivo = '{}.png'.format(str(i).zfill(6))
        url = 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png'
        guardado = '{}{}'.format(ruta, archivo)
    # intenta descarga el archivo, si logra descargarlo marca el error como CERO
    try:
        con = urllib.request.urlretrieve(url, guardado)
        df.at[i, 'error'] = 0
    # si hubo error al descargar, suma 1 al registro de error de esa imagen
    except urllib.error.HTTPError as err:
        df.at[i, 'error'] = df.at[i, 'error'] + 1
    # minima demora para no saturar la conexion (probar el valor mas efectivo)
    time.sleep(0.15)
    return None

# Guarda imagenes en la carpeta iterando por el dataframe
# (iterar no es lo mejor en pandas, pero son pocas operaciones e igualmente deben tener algo de demora)
with tqdm(total=len(df.index)) as pbar:
    for i, row in df.iterrows():
        # en cada iteracion llama a la funcion que descarga la imagen
        url_to_imagen(i, row['profile_image_url'], carpeta_salida)
        pbar.update(1)

# Suma los errores obtenidos
# errores = df.loc[df['error'] > 0].index[0]

# Avisa si hubo errores al descargar imagenes
if df.loc[df['error'] > 0] > 0:
    print ('\n')
    print ('Errores:', df.loc[df['error'] > 0].index[0])
    print (df.loc[df['error'] > 0])
else:
    print ('Todas las imagenes se descargaron sin errores')

100%|██████████| 3000/3000 [12:54<00:00,  3.88it/s]


IndexError: index 0 is out of bounds for axis 0 with size 0

Las pastillas siguientes no son necesarias, son accesorias

In [ ]:
# Descarga de imagenes que no se pudieron bajar por error
# Escribir codigo que establezca una cantidad de veces a probar, y que itere por el dataframe 2 en las filas con error > a cero
# luego de todos los intentos, a las que no se pudieron descargar se les asigna la imagen por defecto
url_to_imagen(2459, 'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', carpeta_salida)


In [ ]:
# -----------------------------------------------------------------------------------------------------------
# IMPORTANTE: Esto no es necesario, es para borrar la carpeta imagenes completa por si hace falta probar algo

shutil.rmtree('imagenes')
# -----------------------------------------------------------------------------------------------------------

In [ ]:
# -----------------------------------------------------------------------------------------------------------
# IMPORTANTE: Esto no es necesario ejecutarlo, es por si hace falta descargar las imagenes, es mas rapido en un solo archivo comprimido grande.
# Comprime carpeta con imagenes y guardar el archivo para poder descargar

shutil.make_archive('fotos', 'zip', carpeta_salida)
# -----------------------------------------------------------------------------------------------------------